In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from orderedset import OrderedSet
from copy import deepcopy
%matplotlib inline  

In [35]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self, rootpath=False):
        subclasses = self.get_subclasses(rootpath=False)
        return [(b,a,c,d) for (a,b,c,d) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        obj_props = [(prop, "Object Property") for prop in self.object_properties]
        data_props = [(prop, "Datatype Property") for prop in self.data_properties]
        props = obj_props + data_props
        all_triples = []
        for prop, prop_type in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop), prop_type) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop), prop_type))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples(rootpath))
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True, rootpath=False):
        return self.parse_triples(union_flag, subclass_of, rootpath)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of", "Subclass"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0]), "Object Property"))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Object Property"))
                    except:
                        try:
                            if not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                            elif not self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                                prop_vals = self.get_child_node(prop[0], "owl:DatatypeProperty")
                                class_vals = self.get_child_node(some_vals[0], "owl:Class")
                                subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0]), "Datatype Property"))
                        except Exception as e:
                            print (e)
                            continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of", "Subclass"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of", "Subclass") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
    
    def extract_ns(self):
        '''
        Extracts namespace of an ontology
        '''
        ns = self.ontology_obj.getElementsByTagName("rdf:RDF")[0].getAttribute("xmlns")
        if ns[-1] == "#":
            return ns
        return self.ontology_obj.doctype.entities.item(0).firstChild.nodeValue
    
    def get_subclasses(self, rootpath=False):
        subclasses = [(self.extract_ID(a, not rootpath), self.extract_ID(b, not rootpath), c, d) for (a,b,c,d) in self.subclasses]
        self.parents_dict = {}
        for (a,b,c,d) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [36]:
alignment_folder = "german_datasets/mapping freizeit/"

ontologies_in_alignment = []

# Load reference alignments 
def load_alignments(folder):
    gt = []
    path = [folder + l for l in os.listdir(folder) if l.endswith(".txt")][0]
    mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
    for mapping in mappings:
        src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
        targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
        ontologies_in_alignment.append((folder + src, folder + targ))
        src = src.rsplit(".",1)[0].replace(".", "_").lower()
        targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
        lines = [["_".join(row.strip().split(":")[0].replace(",-", "_").split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                 for line in mapping.split("\n") if line.startswith(" -")]
        lines = [[src + "#" + line[0], targ + "#" + line[1]] for line in lines]
        gt.append(lines)
    return gt

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)


reference_alignments = load_alignments(alignment_folder)


In [49]:
reference_alignments

[[['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Artistik',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Artistik'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Kerzen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Kerzen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Malen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Malen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Papier',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Papier'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Perlen',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Perlen'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Serviettentechnik',
   'google_freizeit#World_Deutsch_Online-Shops_Freizeit_Basteln_Serviettentechnik'],
  ['dmoz_freizeit#Top_World_Deutsch_Online-Shops_Freizeit_Basteln_Stempeltechnik',
   'google_frei

In [18]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology(l[0])
    ont2 = Ontology(l[1])
    
    ent1 = ont1.get_classes()
    ent2 = ont2.get_classes()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))
    
    pre1 = l[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    pre2 = l[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    print (pre1, pre2)
    all_mappings.extend([(pre1 + "#" + el[0], pre2 + "#" + el[1]) for el in mappings])
    

data = {mapping: False for mapping in all_mappings}
reference_alignments = [tuple(alignment) for alignment in reference_alignments]
s = set(all_mappings)
for mapping in set(reference_alignments):
    if mapping in s:
        data[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data[mapping] = True
        else:
            print (mapping)

dmoz_freizeit google_freizeit


In [48]:
def write_results(final_list):
    ont_name_parsed1 = Ontology(ont_name1).extract_ns()
    ont_name_parsed2 = Ontology(ont_name2).extract_ns()
    ont_name1_pre = ont_name1 if (ont_name1.startswith("http://") or ont_name1.startswith("https://")) else "file://" + ont_name1
    ont_name2_pre = ont_name2 if (ont_name2.startswith("http://") or ont_name2.startswith("https://")) else "file://" + ont_name2
    rdf = \
    """<?xml version='1.0' encoding='utf-8' standalone='no'?>
<rdf:RDF xmlns='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'
         xmlns:rdf='http://www.w3.org/1999/02/22-rdf-syntax-ns#'
         xmlns:xsd='http://www.w3.org/2001/XMLSchema#'
         xmlns:align='http://knowledgeweb.semanticweb.org/heterogeneity/alignment#'>
<Alignment>
  <xml>yes</xml>
  <level>0</level>
  <type>**</type>
  <onto1>
    <Ontology rdf:about="{}">
      <location>{}</location>
    </Ontology>
  </onto1>
  <onto2>
    <Ontology rdf:about="{}">
      <location>{}</location>
    </Ontology>
  </onto2>""".format(ont_name_parsed1.split("#")[0], ont_name1_pre, ont_name_parsed2.split("#")[0], ont_name2_pre)
    for (a,b,score) in final_list:
        mapping = """
  <map>
    <Cell>
      <entity1 rdf:resource='{}'/>
      <entity2 rdf:resource='{}'/>
      <relation>=</relation>
      <measure rdf:datatype='http://www.w3.org/2001/XMLSchema#float'>{}</measure>
    </Cell>
  </map>""".format(ont_name_parsed1 + "#".join(a.split("#")[1:]), ont_name_parsed2 + "#".join(b.split("#")[1:]), score)
        rdf += mapping
    rdf += """
</Alignment>
</rdf:RDF>"""
    return rdf


# Load reference alignments 
def load_alignments(folder):
    gt = []
    path = [folder + l for l in os.listdir(folder) if l.endswith(".txt")][0]
    mappings = [content.strip() for content in open(path).read().split("--------------------------------------------------------") if content.strip()]
    for mapping in mappings:
        src = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Source: ")][0]
        targ = [line.split(":")[-1].strip() for line in mapping.split("\n") if line.startswith(" + Target: ")][0]
        ontologies_in_alignment.append((folder + src, folder + targ))
        src = src.rsplit(".",1)[0].replace(".", "_").lower()
        targ = targ.rsplit(".",1)[0].replace(".", "_").lower()
        lines = [["_".join(row.strip().split(":")[0].replace(",-", "_").split(".")) for row in line.split("-",1)[1].strip().split("<->")]
                 for line in mapping.split("\n") if line.startswith(" -")]
        lines = [[src + "#" + line[0], targ + "#" + line[1], 1.0] for line in lines]
        gt.append(lines)
    return gt

alignment_folder = "german_datasets/mapping webdirectory/"
ontologies_in_alignment = []

final_list = load_alignments(alignment_folder)

ontologies_in_alignment = [('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl'),
 ('/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl',
  '/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl')]

for i,ont_pair in enumerate(ontologies_in_alignment):
    ont_name1, ont_name2 = ont_pair[0], ont_pair[1]
    rdf = write_results(final_list[i])
    c = "_".join(ont_name1.split("/")[-1].split(".")[:-1]) + "-" + "_".join(ont_name2.split("/")[-1].split(".")[:-1]) + ".rdf"
    f = open("/data/Vivek/IBM/VeeAlign/datasets/web-directory/alignments/" + c, "w+")
    f.write(rdf)
    f.close()
    
    
    

In [2]:


def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_") if el])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_") if el])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings([parse(el) for el in keys])))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


NameError: name 'all_mappings' is not defined

In [4]:

filtered_dict = {}
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

# Abbrevation resolution preprocessing
def parse(word):
    return " ".join(flatten([el.split("_") for el in camel_case_split(word)]))

# Extracting USE embeddings

def extractUSEEmbeddings(words):
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3?tf-hub-format=compressed")
    embeds = model(words)
    return embeds.numpy()

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

extracted_elems = []
mapping_ont = {}
for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology(ont_name)
    entities = ont.get_classes()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    ont_name = ont_name.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    mapping_ont[ont_name] = ont
    extracted_elems.extend([ont_name + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp = []
for word in extracted_elems:
    ont_name = word.split("#")[0]
    elem = word.split("#")[1]
    inp.append(parse(mapping_ont[ont_name].mapping_dict.get(elem, elem)))

print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

embeds = np.array(extractUSEEmbeddings(inp))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))


emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


Total number of extracted unique classes and properties from entire RA set:  3032


In [58]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append([e2])
            neighbours_dict[e2][2].append([e1])
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append([e2])
            neighbours_dict[e2][3].append([e1])
        elif d == "Subclass":
            neighbours_dict[e2][1].append([e1])
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    
    for entity in neighbours_dict:
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl


In [6]:
data, _, _, _, neighbours_dicts = pickle.load(open("../Input/data_webdir_bagofnbrs.pkl", "rb"))
f = open("../Input/data_webdir_bagofnbrsv2.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)


In [61]:
def path_to_root(elem, ont_mappings, curr = [], rootpath=[]):
    curr.append(elem)
    if elem not in ont_mappings or not ont_mappings[elem]:
        rootpath.append(curr)
        return
    for node in ont_mappings[elem]:
        curr_orig = deepcopy(curr)
        _ = path_to_root(node, ont_mappings, curr, rootpath)
        curr = curr_orig
    return rootpath

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples(rootpath=True)
    entities = [(a,b) for (a,b,c,d) in triples]
    neighbours_dict = {elem: [[] for i in range(4)] for elem in list(set(flatten(entities)))}
    print (ont)
    for (e1, e2, p, d) in triples:
        if e1==e2:
            continue
        if d == "Object Property":
            neighbours_dict[e1][2].append(e2)
            neighbours_dict[e2][2].append(e1)
        elif d == "Datatype Property":
            neighbours_dict[e1][3].append(e2)
            neighbours_dict[e2][3].append(e1)
        elif d == "Subclass":
            neighbours_dict[e2][1].append(e1)
        else:
            print ("Error wrong value of d: ", d)
    
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict_new = {}
    for elem in rootpath_dict:
#         print ("Done for ", elem)
        rootpath_dict_new[elem] = path_to_root(elem, rootpath_dict, [], [])
    ont = ont.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()

    for entity in neighbours_dict:
        neighbours_dict[entity][1] = [neighbours_dict[entity][1]]
        neighbours_dict[entity][2] = [neighbours_dict[entity][2]]
        neighbours_dict[entity][3] = [neighbours_dict[entity][3]]
        if entity in rootpath_dict_new and len(rootpath_dict_new[entity]) > 0:
            neighbours_dict[entity][0].extend(rootpath_dict_new[entity])
        else:
            continue
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

#     neighbours_dict = {elem: [key[:1] + sorted(list(set(key[1:]))) for key in neighbours_dict[elem]]
#                        for elem in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [[tuple([ont + "#" + node for node in path]) for path in nbr_type]
                                        for nbr_type in neighbours_dict[el]] 
                       for el in neighbours_dict}
    neighbours_dict = {el: [[list(path) for path in nbr_type] for nbr_type in neighbours_dict[el]]
                       for el in neighbours_dict}
    return neighbours_dict

# neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
neighbours_dicts = {}
for ont in list(set(flatten(ontologies_in_alignment))):
    neighbours_dicts = {**neighbours_dicts, **get_one_hop_neighbours(ont)}
max_types = np.max([len([nbr_type for nbr_type in elem if nbr_type]) for elem in neighbours_dicts.values()])
max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)

# max_paths = np.max([[len(nbr_type) for nbr_type in elem] for elem in neighbours_dicts.values()])
# max_pathlen = np.max(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in neighbours_dicts.values()]), axis=0)
# neighbours_dicts_lenpadded = {elem: [[path + ["<UNK>" for i in range(max_pathlen -len(path))] for path in nbr_type]
#                                 for nbr_type in neighbours_dicts[elem]] for elem in neighbours_dicts}
# neighbours_dicts_pathpadded = {elem: [nbr_type + [["<UNK>" for j in range(max_pathlen)] for i in range(max_paths - len(nbr_type))]
#                                 for k,nbr_type in enumerate(neighbours_dicts_lenpadded[elem])] for elem in neighbours_dicts_lenpadded}
# neighbours_dicts_pathpadded = {elem: np.array(neighbours_dicts_pathpadded[elem]) for elem in neighbours_dicts_pathpadded}
# data_items = np.array(list(data.items()))

# data_shuffled_t = [elem for elem in data_items if elem[1]]
# data_shuffled_f = [elem for elem in data_items if not elem[1]]
# np.random.shuffle(data_items)
# data_shuffled_f = data_shuffled_f[:150000-len(data_shuffled_t)]

# indices = np.random.permutation(len(data_shuffled_t) + len(data_shuffled_f[:130000-len(data_shuffled_t)]))

# data_shuffled = data_shuffled_t + data_shuffled_f
# indices = np.random.permutation(len(data_shuffled))

# indices = np.random.permutation(len(data_shuffled))

# data = OrderedDict(data_items)

# ontologies_in_alignment_rev = [[el.split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() for el in ont] for ont in ontologies_in_alignment]

# f = open("Input/data_freizeit_bagofnbrs.pkl", "wb")
# pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)
# f = open("data_aml_uniqpath.pkl", "wb")
# pickle.dump([data, aml_data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts_pathpadded, ontologies_in_alignment], f)
# # # # # neighbours_dicts

/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/yahoo.small.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/google.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/dmoz.owl
/data/Vivek/IBM/VeeAlign/datasets/web-directory/ontologies/web.owl


In [77]:
leb_data = pickle.load(open("Input/data_freizeit.pkl", "rb"))[0]
data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts = pickle.load(open("Input/data_webdir.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("Input/data_fre_webdir_bagofnbrs.pkl", "wb"))

In [19]:
leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, _ = pickle.load(open("../Input/data_webdir_leb.pkl", "rb"))
pickle.dump([leb_data, data, emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], open("../Input/data_webdir_leb_bagofnbrs.pkl", "wb"))
# pickle.dump([pickle.load(open("Input/data_freizeit.pkl", "rb"))[0],
#              pickle.load(open("../Input/data_webdir.pkl", "rb"))[0],
#              emb_indexer, emb_indexer_inv, emb_vals, neighbours_dicts], f)

In [59]:
neighbours_dicts

{'yahoo_small#Firmen_Sport_Motorsport': [[['yahoo_small#Firmen_Sport_Motorsport',
    'yahoo_small#Firmen_Sport',
    'yahoo_small#Firmen']],
  [['yahoo_small#Firmen_Sport_Motorsport_Fanartikel'],
   ['yahoo_small#Firmen_Sport_Motorsport_Kartsport']],
  [],
  []],
 'yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit_Bildung-und-Ausbildung': [[['yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit_Bildung-und-Ausbildung',
    'yahoo_small#Firmen_Gesundheit_Alternativmedizin_Atemarbeit',
    'yahoo_small#Firmen_Gesundheit_Alternativmedizin',
    'yahoo_small#Firmen_Gesundheit',
    'yahoo_small#Firmen']],
  [],
  [],
  []],
 'yahoo_small#Firmen_Sport_Radsport_Mountain-Biking': [[['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking',
    'yahoo_small#Firmen_Sport_Radsport',
    'yahoo_small#Firmen_Sport',
    'yahoo_small#Firmen']],
  [['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking_Komponenten'],
   ['yahoo_small#Firmen_Sport_Radsport_Mountain-Biking_Hersteller']],
  [],

In [62]:
np.max([len([nbr_type for nbr_type in elem if flatten(nbr_type)]) for elem in neighbours_dicts.values()])

2

In [27]:
# max_paths
sorted(Counter(flatten([[len(nbr_type) for nbr_type in elem] for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())


[(0, 8658),
 (1, 3486),
 (2, 148),
 (3, 99),
 (4, 69),
 (5, 33),
 (6, 32),
 (7, 37),
 (8, 12),
 (9, 11),
 (10, 6),
 (11, 6),
 (12, 8),
 (13, 7),
 (15, 4),
 (16, 3),
 (17, 3),
 (18, 1),
 (19, 2),
 (20, 4),
 (21, 5),
 (22, 3),
 (23, 1),
 (24, 2),
 (26, 3),
 (38, 1),
 (41, 1),
 (42, 1),
 (48, 1),
 (49, 1)]

In [28]:
sorted(Counter(flatten([flatten([[len(path) for path in nbr_type] for nbr_type in elem]) for elem in pickle.load(open("../Input/data_webdir_fre.pkl", "rb"))[-1].values()])).items())

[(1, 3156),
 (2, 25),
 (3, 426),
 (4, 1243),
 (5, 1054),
 (6, 329),
 (7, 53),
 (8, 26)]

In [ ]:
embedding_dim = np.array(emb_vals).shape[1]
        
name_embedding = nn.Embedding(len(emb_vals), embedding_dim)
name_embedding.load_state_dict({'weight': torch.from_numpy(np.array(emb_vals))})
name_embedding.weight.requires_grad = False

cosine_sim_layer = nn.CosineSimilarity(dim=1)
output = nn.Linear(2*embedding_dim, 300)
max_pathlen = np.array(list(neighbours_dicts_pathpadded.values())).shape[3]
v = nn.Parameter(torch.DoubleTensor([1/(max_pathlen) for i in range(max_pathlen)]))

nodes = torch.randint(0, len(emb_vals), size=(10,2))
features= torch.randint(0, len(emb_vals), size=(10,2,4,22,6))

nodes = nodes.permute(1,0) # 2 * batch_size
features = features.permute(1,0,2,3,4) # 2 * batch_size * 4 * max_paths * max_pathlen
for i in range(2):
    node_emb = name_embedding(nodes[i]) # batch_size * 512
    feature_emb = name_embedding(features[i]) #  batch_size * 4 * max_paths * max_pathlen * 512

    path_weights = torch.sum(torch.sum(node_emb[:, None, None, None, :] * feature_emb, dim=-1), dim=-1)
    best_path_indices = torch.max(path_weights, dim=2)[1][(..., ) + (None, ) * 3]
    best_path_indices = best_path_indices.expand(-1, -1, -1, max_pathlen,  embedding_dim)
    best_path = torch.gather(feature_emb, 2, best_path_indices).squeeze(2) # batch_size * 4 * max_pathlen * 512
    # Another way: 
    # path_weights = masked_softmax(path_weights)
    # best_path = torch.sum(path_weights.unsqueeze(-1) * feature_emb, dim=2)

    node_weights = torch.sum(node_emb[:, None, None, :] * best_path, dim=-1).unsqueeze(-1)
    torch.matmul(v, node_weights * best_path)

In [ ]:
def masked_softmax(inp):
    inp = inp.double()
    mask = ((inp != 0).double() - 1) * 9999  # for -inf
    return (inp + mask).softmax(dim=-1)
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))

attended_path = torch.bmm(a.reshape(-1, 1, 512), b.reshape(-1, 512, 1))
attended_path = attended_path.reshape(-1, 4, 22, 6)
path_weights = masked_softmax(torch.sum(attended_path, dim=-1))
path_weights.shape
# best_path = torch.sum(path_weights[:, :, :, None, None] * a, dim=2)

In [ ]:
indices = d[(..., ) + (None, ) * 3].expand(-1, -1, -1, 6, 512)
e = torch.gather(b, 2, indices).squeeze(2)

f = torch.sum(a[:,None,None,:] *e,dim=-1).unsqueeze(-1)

g = (f*e)

h = torch.sum((v[None,None,:,None] * g), dim=2)

In [ ]:
c = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,1,6,512))
d = b * c


In [ ]:
e = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)


In [ ]:
fo

In [24]:
len(data)

3318178

In [ ]:
ctimes, dtimes = [], []
for i in range(100):
    a = torch.randn((10,4,22,6,512))
    b = torch.randn((10,4,1,6,512))
    t = time.time()
    c = a * b
    ctimes.append(time.time()-t)
    t = time.time()
    d = torch.bmm(c.permute(0,1,3,4,2).reshape(-1, 22, 1), b.permute(0,1,3,4,2).reshape(-1, 1, 1)).squeeze(-1).reshape(-1,4,6,512,22).permute(0,1,4,2,3)
    dtimes.append(time.time()-t)

In [ ]:
a = torch.randn((10,4,22,6,512))
b = torch.randn((10,4,22,6,512))
c = a * torch.sum(b,dim=2).unsqueeze(2)

In [23]:
arr = [('german_datasets_copy/lebensmittel/Google.Lebensmittel.owl',
  'german_datasets_copy/lebensmittel/web.Lebensmittel.owl')]
for ont_pair in arr:
    a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
    !rm -rf $c
    os.mkdir(c)
    java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a) + \
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
    process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
pred_logmap = [[el.split("/")[-1] for el in l.split("\t")[:-1]] for l in open(os.path.abspath(c + "/") + "/logmap2_mappings.tsv",  "r").read().split("\n")[:-1] if not l.startswith("Optional")]


KeyboardInterrupt: 

In [ ]:
gt_mappings = []
for elem in pred_logmap:
    gt_mappings.append(tuple([el.split("#")[0].replace(".v2", "").rsplit(".",1)[0].replace(".", "_").lower() + "#" + el.split("#")[1] for el in elem]))


In [ ]:
data_orig = pickle.load(open("../Input/data_lebensmittel.pkl", "rb"))[0]
data_logmap = {}
for key in data_orig:
    data_logmap[key] = False
s = set(list(data_logmap.keys()))
gt_mappings = [tuple(pair) for pair in gt_mappings]
for mapping in gt_mappings:
    
    if mapping in s:
        data_logmap[mapping] = True
    else:
        mapping = tuple([el.replace(",-", "_") for el in mapping])
        if mapping in s:
            data_logmap[mapping] = True
        else:
            print (mapping)

In [ ]:
all_metrics = []
def return_test_data(data, i):
    data_t = {elem: data[elem] for elem in data if data[elem]}
    data_f = {elem: data[elem] for elem in data if not data[elem]}

    data_t_items = list(data_t.keys())
    data_f_items = list(data_f.keys())

    test_data_t = data_t_items[int((0.2*i)*len(data_t)):int((0.2*i + 0.2)*len(data_t))]
    test_data_f = data_f_items[int((0.2*i)*len(data_f)):int((0.2*i + 0.2)*len(data_f))]
    
    test_data = {}
    for elem in test_data_t:
        test_data[elem] = True
    for elem in test_data_f:
        test_data[elem] = False
    return test_data

for i in range(5):
    test_gt = return_test_data(data_orig, i)
    test_logmap = {elem: data_logmap[elem] for elem in test_gt}
    tp = len([elem for elem in test_gt if test_gt[elem] and test_logmap[elem]])
    fp = len([elem for elem in test_logmap if not test_gt[elem] and test_logmap[elem]])
    fn = len([elem for elem in test_logmap if test_gt[elem] and not test_logmap[elem]])
    
    try:
        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1score = 2 * precision * recall / (precision + recall)
        f2score = 5 * precision * recall / (4 * precision + recall)
        f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    except Exception as e:
        print (e)
        continue
    all_metrics.append((precision, recall, f1score, f2score, f0_5score))
    

In [ ]:
np.mean(all_metrics, axis=0)

In [ ]:
# AML test
def is_test(test_onto, key):
    return tuple([el.split("#")[0] for el in key]) in test_onto

results = []

for i in list(range(0, ontologies_in_alignment, 3)):
    test_onto = all_ont_pairs[i:i+3]
    for ont_pair in test_onto:
        print (ont_pair)
        a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower() + "-" + ont_pair[1].split("/")[-1].rsplit(".",1)[0].replace(".", "_").lower()
        java_command = "java -jar logmap-matcher/target/logmap-matcher-4.0.jar MATCHER file:" +  os.path.abspath(a)
                     " file:" + os.path.abspath(b) + " " + "/data/Vivek/IBM/IBM-Internship/" + c + "/ false"
        process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
    print (os.listdir("AML-test-results/"))
    pred_aml = load_alignments("AML-test-results/")
    pred_aml = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml]
    tp = len([elem for elem in pred_aml if data[elem]])
    fn = len([key for key in gt_mappings if key not in set(pred_aml) and is_test(test_onto, key)])
    fp = len([elem for elem in pred_aml if not data[elem]])

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1score = 2 * precision * recall / (precision + recall)
    f2score = 5 * precision * recall / (4 * precision + recall)
    f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
    print (precision, recall, f1score, f2score, f0_5score)
    
    metrics = [precision, recall, f1score, f2score, f0_5score]
    results.append(metrics)
    
    _ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

In [ ]:
ontologies_in_alignment = [[el.split("/")[1].split(".")[0] for el in ont] for ont in ontologies_in_alignment][:-1] + [["human", "mouse"]]

In [14]:
output = """Output_att13_4_0.66_conf_oaei_german_aml.txt:Final Results: [0.73893805 0.54754098 0.62900188 0.57745505 0.69065343]
Output_att13_4_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_4_0.735_conf_oaei_german_aml.txt:Final Results: [0.73390558 0.56065574 0.63568773 0.58843772 0.69118836]
Output_att13_4_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_4_0.736_conf_oaei_german_aml.txt:Final Results: [0.72340426 0.55737705 0.62962963 0.58419244 0.68273092]
Output_att13_4_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_4_0.8_conf_oaei_german_aml.txt:Final Results: [0.784689   0.53770492 0.6381323  0.57382785 0.71866784]
Output_att13_4_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.67234043 0.51803279 0.58518519 0.54295533 0.63453815]
Output_att13_4_0.96_conf_oaei_german_aml.txt:Final Results: [0.68461538 0.58360656 0.6300885  0.60135135 0.66171004]
Output_att13_4_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.75862069 0.50491803 0.60629921 0.54111033 0.68934646]
Output_att13_4_0.98_conf_oaei_german_aml.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att13_4_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.6981982  0.50819672 0.58823529 0.53744799 0.6496228 ]
Binary file Output_att13_4_conf_oaei_german_aml.txt matches
Binary file Output_att13_4_conf_oaei_german_aml_wtpath.txt matches
Output_att13_5_0.66_conf_oaei_german_aml.txt:Final Results: [0.72862454 0.64262295 0.68292683 0.65815984 0.7096307 ]
Output_att13_5_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_5_0.735_conf_oaei_german_aml.txt:Final Results: [0.73251029 0.58360656 0.64963504 0.60833903 0.69694597]
Output_att13_5_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_5_0.736_conf_oaei_german_aml.txt:Final Results: [0.74008811 0.55081967 0.63157895 0.5805114  0.69249794]
Output_att13_5_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_5_0.8_conf_oaei_german_aml.txt:Final Results: [0.75943396 0.52786885 0.62282398 0.56215084 0.69817866]
Output_att13_5_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.75980392 0.50819672 0.60903733 0.54424157 0.69134701]
Output_att13_5_0.96_conf_oaei_german_aml.txt:Final Results: [0.68093385 0.57377049 0.6227758  0.59241706 0.6564141 ]
Output_att13_5_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.60573477 0.55409836 0.57876712 0.56370914 0.59465165]
Output_att13_5_0.98_conf_oaei_german_aml.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att13_5_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.58781362 0.53770492 0.56164384 0.54703135 0.57705841]
Binary file Output_att13_5_conf_oaei_german_aml.txt matches
Binary file Output_att13_5_conf_oaei_german_aml_wtpath.txt matches
Output_att13_8_0.66_conf_oaei_german_aml.txt:Final Results: [0.75       0.55081967 0.63516068 0.58171745 0.69941715]
Output_att13_8_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.64777328 0.52459016 0.57971014 0.54533061 0.61871616]
Output_att13_8_0.735_conf_oaei_german_aml.txt:Final Results: [0.71272727 0.64262295 0.67586207 0.65551839 0.6975089 ]
Output_att13_8_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.62645914 0.52786885 0.57295374 0.5450237  0.60390098]
Output_att13_8_0.736_conf_oaei_german_aml.txt:Final Results: [0.73478261 0.55409836 0.6317757  0.58275862 0.68979592]
Output_att13_8_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.51311953 0.57704918 0.54320988 0.56301983 0.52474657]
Output_att13_8_0.8_conf_oaei_german_aml.txt:Final Results: [0.77386935 0.50491803 0.61111111 0.54263566 0.69936421]
Output_att13_8_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att13_8_0.96_conf_oaei_german_aml.txt:Final Results: [0.73684211 0.55081967 0.630394   0.5801105  0.69022186]
Output_att13_8_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76119403 0.50163934 0.60474308 0.53835327 0.68981064]
Output_att13_8_0.98_conf_oaei_german_aml.txt:Final Results: [0.75490196 0.50491803 0.60510806 0.54073034 0.68688671]
Output_att13_8_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Binary file Output_att13_8_conf_oaei_german_aml.txt matches
Binary file Output_att13_8_conf_oaei_german_aml_wtpath.txt matches
Binary file Output_att1_3_conf_oaei_german_aml.txt matches
Binary file Output_att1_3_conf_oaei_german_aml_wtpath.txt matches
Binary file Output_att1_4_conf_oaei_german_aml_wtpath.txt matches
Binary file Output_att1_5_conf_oaei_german_aml_wtpath.txt matches
Binary file Output_att1_8_conf_oaei_german_aml.txt matches
Binary file Output_att1_8_conf_oaei_german_aml_wtpath.txt matches
Output_att2_10_conf_oaei_aml_bagofnbrs.txt:Final Results: [0.62162162 0.49640288 0.552      0.51724138 0.59176672]
Output_att2_12_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76303318 0.52786885 0.62403101 0.56254368 0.70060923]
Output_att2_12_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.62681159 0.56721311 0.59552496 0.57820856 0.61391057]
Output_att2_12_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73029046 0.57704918 0.64468864 0.60232717 0.69345942]
Output_att2_12_0.735_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.65991903 0.53442623 0.59057971 0.55555556 0.63031709]
Output_att2_12_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73504274 0.56393443 0.63821892 0.5914718  0.69298952]
Output_att2_12_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.59851301 0.52786885 0.56097561 0.5406313  0.58291093]
Output_att2_12_0.8_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_12_0.8_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_12_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.70682731 0.57704918 0.63537906 0.59904697 0.67640277]
Output_att2_12_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.74879227 0.50819672 0.60546875 0.54309741 0.68402471]
Output_att2_12_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_12_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.75609756 0.50819672 0.60784314 0.54385965 0.68888889]
Output_att2_12_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77664975 0.50163934 0.60956175 0.53987297 0.69990851]
Output_att2_12_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.75609756 0.50819672 0.60784314 0.54385965 0.68888889]
Output_att2_12_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76442308 0.52131148 0.61988304 0.55672269 0.69920844]
Output_att2_12__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.64285714 0.53114754 0.58168761 0.55027174 0.61690784]
Output_att2_12_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_12_conf_oaei_german_bagofnbrs.txt:Final Results: [0.0053779  0.99489796 0.01069284 0.02627135 0.00671249]
Output_att2_12_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00588588 1.         0.01169715 0.02869791 0.00734563]
Output_att2_13_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.70703125 0.59344262 0.64527629 0.61314363 0.68096313]
Output_att2_13_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.48541114 0.6        0.53665689 0.57294928 0.50468836]
Output_att2_13_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.72874494 0.59016393 0.65217391 0.61349693 0.69605568]
Output_att2_13_0.735_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_13_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76525822 0.53442623 0.62934363 0.56873692 0.70440795]
Output_att2_13_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.5451505  0.53442623 0.5397351  0.5365372  0.54297135]
Output_att2_13_0.8_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_13_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75728155 0.51147541 0.61056751 0.54698457 0.69087688]
Output_att2_13_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.46015424 0.58688525 0.51585014 0.55624612 0.48092423]
Output_att2_13_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.74519231 0.50819672 0.6042885  0.54271709 0.68161829]
Output_att2_13_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.58116883 0.58688525 0.58401305 0.58573298 0.58230319]
Output_att2_13_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_13_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_13_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.6996337  0.62622951 0.66089965 0.63965171 0.68360773]
Output_att2_13__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.58122744 0.52786885 0.5532646  0.53774215 0.56970984]
Binary file Output_att2_13_conf_oaei_german_aml_bagofnbrs_wtpath.txt matches
Binary file Output_att21_3_conf_oaei_german_aml.txt matches
Binary file Output_att21_3_conf_oaei_german_aml_wtpath.txt matches
Output_att21_4_0.66_conf_oaei_german_aml.txt:Final Results: [0.71936759 0.59672131 0.65232975 0.61778683 0.69096431]
Output_att21_4_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.69620253 0.54098361 0.60885609 0.56623198 0.65841979]
Output_att21_4_0.735_conf_oaei_german_aml.txt:Final Results: [0.70588235 0.6295082  0.66551127 0.64343164 0.68916009]
Output_att21_4_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.51376147 0.55081967 0.53164557 0.54298643 0.52076875]
Output_att21_4_0.736_conf_oaei_german_aml.txt:Final Results: [0.74429224 0.53442623 0.6221374  0.56636553 0.69009314]
Output_att21_4_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.68103448 0.51803279 0.58845438 0.54407713 0.64071371]
Output_att21_4_0.8_conf_oaei_german_aml.txt:Final Results: [0.73214286 0.53770492 0.62003781 0.56786704 0.68276436]
Output_att21_4_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.66386555 0.51803279 0.58195212 0.54183813 0.62848051]
Output_att21_4_0.96_conf_oaei_german_aml.txt:Final Results: [0.75       0.54098361 0.62857143 0.57291667 0.69620253]
Output_att21_4_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.65758755 0.55409836 0.60142349 0.57210562 0.63390848]
Output_att21_4_0.98_conf_oaei_german_aml.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att21_4_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.75362319 0.51147541 0.609375   0.54660126 0.68843778]
Binary file Output_att21_4_conf_oaei_german_aml.txt matches
Binary file Output_att21_4_conf_oaei_german_aml_wtpath.txt matches
Output_att21_5_0.66_conf_oaei_german_aml.txt:Final Results: [0.76923077 0.52459016 0.62378168 0.56022409 0.70360598]
Output_att21_5_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att21_5_0.735_conf_oaei_german_aml.txt:Final Results: [0.69144981 0.60983607 0.64808362 0.62458026 0.67342505]
Output_att21_5_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.68722467 0.51147541 0.58646617 0.5390463  0.6430338 ]
Output_att21_5_0.736_conf_oaei_german_aml.txt:Final Results: [0.70940171 0.5442623  0.61595547 0.57083906 0.66881547]
Output_att21_5_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.61565836 0.56721311 0.59044369 0.57628248 0.6053184 ]
Output_att21_5_0.8_conf_oaei_german_aml.txt:Final Results: [0.75586854 0.52786885 0.62162162 0.56175855 0.69576491]
Output_att21_5_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.73684211 0.50491803 0.59922179 0.53883835 0.67484663]
Output_att21_5_0.96_conf_oaei_german_aml.txt:Final Results: [0.765      0.50163934 0.60594059 0.53873239 0.69230769]
Output_att21_5_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.6826087  0.5147541  0.58691589 0.54137931 0.64081633]
Output_att21_5_0.98_conf_oaei_german_aml.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att21_5_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.59219858 0.54754098 0.56899489 0.55592543 0.58269365]
Binary file Output_att21_5_conf_oaei_german_aml.txt matches
Binary file Output_att21_5_conf_oaei_german_aml_wtpath.txt matches
Output_att21_8_0.66_conf_oaei_german_aml.txt:Final Results: [0.69803922 0.58360656 0.63571429 0.60338983 0.67169811]
Output_att21_8_0.66_conf_oaei_german_aml_wtpath.txt:Final Results: [0.75862069 0.50491803 0.60629921 0.54111033 0.68934646]
Output_att21_8_0.735_conf_oaei_german_aml.txt:Final Results: [0.65714286 0.60327869 0.62905983 0.61333333 0.64561404]
Output_att21_8_0.735_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att21_8_0.736_conf_oaei_german_aml.txt:Final Results: [0.704      0.57704918 0.63423423 0.59863946 0.6743295 ]
Output_att21_8_0.736_conf_oaei_german_aml_wtpath.txt:Final Results: [0.59340659 0.53114754 0.56055363 0.54253182 0.57981389]
Output_att21_8_0.8_conf_oaei_german_aml.txt:Final Results: [0.76699029 0.51803279 0.6183953  0.55399719 0.69973428]
Output_att21_8_0.8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att21_8_0.96_conf_oaei_german_aml.txt:Final Results: [0.73584906 0.51147541 0.60348162 0.54469274 0.6764961 ]
Output_att21_8_0.96_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att21_8_0.98_conf_oaei_german_aml.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att21_8_0.98_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Binary file Output_att21_8_conf_oaei_german_aml.txt matches
Binary file Output_att21_8_conf_oaei_german_aml_wtpath.txt matches
Output_att2_21_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.69372694 0.61639344 0.65277778 0.63044936 0.67674586]
Output_att2_21_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.68995633 0.51803279 0.5917603  0.54520359 0.64701065]
Output_att2_21_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.7219917  0.5704918  0.63736264 0.59548255 0.6855792 ]
Output_att2_21_0.735_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.60447761 0.53114754 0.56544503 0.54435484 0.58823529]
Output_att2_21_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73893805 0.54754098 0.62900188 0.57745505 0.69065343]
Output_att2_21_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_21_0.8_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75490196 0.50491803 0.60510806 0.54073034 0.68688671]
Output_att2_21_0.8_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76119403 0.50163934 0.60474308 0.53835327 0.68981064]
Output_att2_21_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_21_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.6779661  0.52459016 0.59149723 0.54945055 0.64051241]
Output_att2_21_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75862069 0.50491803 0.60629921 0.54111033 0.68934646]
Output_att2_21_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.75862069 0.50491803 0.60629921 0.54111033 0.68934646]
Output_att2_21_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_21_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.62068966 0.53114754 0.57243816 0.54692775 0.60044477]
Binary file Output_att2_21_conf_oaei_german_aml_bagofnbrs.txt matches
Output_att2_21__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Binary file Output_att2_21_conf_oaei_german_aml_bagofnbrs_wtpath.txt matches
Output_att2_21_conf_oaei_german_bagofnbrs.txt:Final Results: [0.0052564  1.         0.01045345 0.02569885 0.00656119]
Output_att2_21_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00523069 1.         0.01040272 0.02557697 0.00652916]
Output_att2_2_conf_oaei_aml_bagofnbrs.txt:Final Results: [0.6        0.48175182 0.53441296 0.50151976 0.57192374]
Output_att2_2_conf_oaei_aml.txt:Final Results: [0.62380952 0.47810219 0.54132231 0.50153139 0.58797127]
Output_att2_2_conf_oaei_german_bagofnbrs.txt:Final Results: [0.00496605 1.         0.00987952 0.02431308 0.00619931]
Output_att2_2_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00554822 1.         0.01103044 0.02709323 0.00692492]
Output_att2_3_conf_oaei_aml_bagofnbrs.txt:Final Results: [0.62608696 0.52554745 0.57142857 0.54298643 0.60301508]
Output_att2_3_conf_oaei_aml.txt:Final Results: [0.62085308 0.47810219 0.54020619 0.50114767 0.58586762]
Output_att2_3_conf_oaei_german_aml.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_3_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_3_conf_oaei_german.txt:Final Results: [0.00491708 1.         0.00978223 0.02407458 0.0061382 ]
Output_att2_3_conf_oaei_german_wtpath.txt:Final Results: [0.00636204 1.         0.01263694 0.03095724 0.00793886]
Output_att2_4_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76585366 0.5147541  0.61568627 0.55087719 0.69777778]
Output_att2_4_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.52884615 0.54098361 0.53484603 0.53851175 0.53122988]
Output_att2_4_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73451327 0.5442623  0.6252354  0.57399723 0.68651778]
Output_att2_4_0.735_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.75609756 0.50819672 0.60784314 0.54385965 0.68888889]
Output_att2_4_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.74439462 0.5442623  0.62878788 0.57519058 0.69340017]
Output_att2_4_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_4_0.8_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73109244 0.5704918  0.64088398 0.59670782 0.69212411]
Output_att2_4_0.8_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.69777778 0.5147541  0.59245283 0.5432526  0.65145228]
Output_att2_4_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76635514 0.53770492 0.63198459 0.57182706 0.70628768]
Output_att2_4_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_4_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76119403 0.50163934 0.60474308 0.53835327 0.68981064]
Output_att2_4_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_4_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77664975 0.50163934 0.60956175 0.53987297 0.69990851]
Output_att2_4_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_49_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75862069 0.50491803 0.60629921 0.54111033 0.68934646]
Output_att2_49_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_49_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.6988417  0.59344262 0.64184397 0.61189993 0.6748695 ]
Output_att2_49_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.72687225 0.54098361 0.62030075 0.57014513 0.6801319 ]
Output_att2_49_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.71365639 0.53114754 0.60902256 0.55977885 0.66776587]
Output_att2_49_0.8_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75845411 0.5147541  0.61328125 0.55010512 0.69285084]
Output_att2_49_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.75342466 0.54098361 0.62977099 0.5733148  0.69856054]
Output_att2_49_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.6259542  0.53770492 0.57848325 0.55330634 0.60606061]
Output_att2_49_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_49_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.75490196 0.50491803 0.60510806 0.54073034 0.68688671]
Output_att2_49_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76623377 0.38688525 0.51416122 0.4294032  0.64060803]
Output_att2_49_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.69777778 0.5147541  0.59245283 0.5432526  0.65145228]
Binary file Output_att2_49_conf_oaei_german_aml_bagofnbrs.txt matches
Output_att2_49__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Binary file Output_att2_49_conf_oaei_german_aml_bagofnbrs_wtpath.txt matches
Output_att2_4_conf_oaei_aml_bagofnbrs.txt:Final Results: [0.62085308 0.48161765 0.54244306 0.50423403 0.58691756]
Output_att2_4_conf_oaei_aml.txt:Final Results: [0.6372549  0.4676259  0.53941909 0.49392097 0.59414991]
Output_att2_4_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73684211 0.50491803 0.59922179 0.53883835 0.67484663]
Output_att2_4__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_4_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76119403 0.50163934 0.60474308 0.53835327 0.68981064]
Output_att2_4_conf_oaei_german_aml.txt:Final Results: [0.75       0.53114754 0.621881   0.56406685 0.69289991]
Output_att2_4_conf_oaei_german_aml_wtpath.txt:Final Results: [0.61068702 0.52459016 0.5643739  0.53981107 0.59127864]
Output_att2_4_conf_oaei_german_bagofnbrs.txt:Final Results: [0.00579772 1.         0.01152256 0.028274   0.00723571]
Output_att2_4_conf_oaei_german.txt:Final Results: [0.00490633 1.         0.00976118 0.02402523 0.00612483]
Output_att2_4_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00587431 1.         0.01167433 0.02864319 0.00733122]
Output_att2_4_conf_oaei_german_wtpath.txt:Final Results: [0.00429654 1.         0.00855317 0.02108945 0.00536441]
Output_att2_5_conf_oaei_german_aml.txt:Final Results: [0.6557377  0.52459016 0.58287796 0.54644809 0.6245121 ]
Output_att2_5_conf_oaei_german_aml_wtpath.txt:Final Results: [0.66532258 0.54098361 0.59674503 0.5619891  0.63608327]
Output_att2_5_conf_oaei_german_bagofnbrs.txt:Final Results: [0.00417018 1.         0.00830284 0.02048096 0.00520684]
Output_att2_5_conf_oaei_german.txt:Final Results: [0.00563907 0.99489796 0.0112094  0.02752089 0.00703805]
Output_att2_5_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00492273 1.         0.00979356 0.02410308 0.00614527]
Output_att2_5_conf_oaei_german_wtpath.txt:Final Results: [0.00555569 1.         0.01104507 0.02712761 0.0069342 ]
Output_att2_6_0.66_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76666667 0.52786885 0.62524272 0.56293706 0.70305677]
Output_att2_6_0.66_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.64658635 0.52786885 0.58122744 0.54799183 0.6187548 ]
Output_att2_6_0.735_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.70342205 0.60655738 0.65140845 0.62373567 0.6816507 ]
Output_att2_6_0.735_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.65432099 0.52131148 0.58029197 0.54340396 0.62255286]
Output_att2_6_0.736_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.7239819  0.52459016 0.60836502 0.55517002 0.67283431]
Output_att2_6_0.736_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.71945701 0.52131148 0.60456274 0.55170021 0.6686291 ]
Output_att2_6_0.8_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.73617021 0.56721311 0.64074074 0.59450172 0.69477912]
Output_att2_6_0.8_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_6_0.96_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77386935 0.50491803 0.61111111 0.54263566 0.69936421]
Output_att2_6_0.96_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_6_0.98_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77272727 0.50163934 0.6083499  0.53949224 0.69735643]
Output_att2_6_0.98_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_6_0.99_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.77083333 0.4852459  0.59557344 0.52407932 0.68965517]
Output_att2_6_0.99_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_6_conf_oaei_german_aml_bagofnbrs.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_6__conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.65163934 0.52131148 0.57923497 0.54303279 0.6206089 ]
Output_att2_6_conf_oaei_german_aml_bagofnbrs_wtpath.txt:Final Results: [0.73023256 0.5147541  0.60384615 0.54703833 0.67381974]
Output_att2_6_conf_oaei_german_bagofnbrs.txt:Final Results: [0.00513003 1.         0.01020264 0.02508602 0.00640352]
Output_att2_6_conf_oaei_german.txt:Final Results: [0.00536389 1.         0.01066584 0.02621122 0.00669514]
Output_att2_6_conf_oaei_german_wtpath_bagofnbrs.txt:Final Results: [0.00445452 1.         0.00886637 0.02185228 0.00556146]
Output_att2_6_conf_oaei_german_wtpath.txt:Final Results: [0.00474645 1.         0.00944448 0.02325567 0.00592547]
Output_att2_7_conf_oaei_aml.txt:Final Results: [0.62318841 0.46909091 0.53526971 0.49349656 0.58476881]
Output_att2_8_conf_oaei_aml_bagofnbrs.txt:Final Results: [0.60747664 0.47619048 0.5338809  0.49770291 0.57573074]
Output_att2_8_conf_oaei_german_aml.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_8_conf_oaei_german_aml_wtpath.txt:Final Results: [0.76237624 0.50491803 0.60749507 0.54149086 0.6918239 ]
Output_att2_8_conf_oaei_german.txt:Final Results: [0.00426319 1.         0.00848745 0.02093217 0.00532288]
Output_att2_8_conf_oaei_german_wtpath.txt:Final Results: [0.00504953 1.         0.01004414 0.02470772 0.00630329]"""

max([float(l.split(":")[2].strip().split()[2]) if l.startswith("Output") else 0 for l in output.split("\n")])

0.68292683

In [ ]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [ ]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

In [7]:
[1/3 for i in range(3)]

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]

NameError: name 'gt_mappings' is not defined

In [ ]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




In [ ]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

In [ ]:
inp_spellchecked[730], inp[731]

In [ ]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(4)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(4)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)
create_comparison_file("test_v2.pkl", 2)
create_comparison_file("test_resolved.pkl", 3)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

In [ ]:
ontologies_in_alignment = pickle.load(open("data_path.pkl", "rb"))[-1]
ontologies_in_alignment

In [ ]:
d = {('confOf#Organization', 'sigkdd#Organizator'): (1,2,3,4),
 ('iasted#Document', 'sigkdd#Document'): (5,6,78,8)}
[[str(el) for el in flatten(el)] for el in d.items()]

In [ ]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

In [ ]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [ ]:
cos_sim(*extractUSEEmbeddings(["Conference Banquet", "Dinner Banquet"]))

In [ ]:
fn

In [ ]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [ ]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

In [ ]:
Ontology("conference_ontologies/conference.owl").triples

In [ ]:
cos_sim2